# Cellucid Jupyter smoketest

This notebook exercises:
- Jupyter embedding (`show_anndata`)
- hooks (`on_ready`, `on_selection`, `on_hover`, `on_click`)
- connectivity debugging (`viewer.debug_connection()`)
- session capture and AnnData mutation (`viewer.get_session_bundle()` / `viewer.apply_session_to_anndata()`)

Dataset:
- `cellucid-python/data/experiments/he_developmental_complete_with_3d_umap.h5ad`


In [ ]:
from __future__ import annotations

from pathlib import Path

DATA_REL_1 = Path("cellucid-python/data/experiments/he_developmental_complete_with_3d_umap.h5ad")
DATA_REL_2 = Path("data/experiments/he_developmental_complete_with_3d_umap.h5ad")

for candidate in (DATA_REL_1, DATA_REL_2):
    if candidate.exists():
        DATA_PATH = candidate.resolve()
        break
else:
    raise FileNotFoundError(
        "Could not find he_developmental_complete_with_3d_umap.h5ad. "
        "Expected at cellucid-python/data/experiments/he_developmental_complete_with_3d_umap.h5ad"
    )

DATA_PATH

In [ ]:
from cellucid import show_anndata

viewer = show_anndata(str(DATA_PATH), height=650)

@viewer.on_ready
def _on_ready(event):
    print("[ready]", event)

@viewer.on_selection
def _on_selection(event):
    print("[selection]", {k: (len(v) if k == "cells" and isinstance(v, list) else v) for k, v in event.items()})

@viewer.on_hover
def _on_hover(event):
    # hover is frequent; keep it compact
    cell = event.get("cell")
    if cell is not None:
        print("[hover]", cell)

@viewer.on_click
def _on_click(event):
    print("[click]", event)

viewer.wait_for_ready(timeout=60)
viewer.state

In [ ]:
# Connectivity/debug report (includes server probes + ping/pong + forwarded console errors)
report = viewer.debug_connection(timeout=10)
report

In [ ]:
# Session capture + apply to AnnData (no manual browser download)
#
# 1) Make some highlights + user-defined labels in the UI
# 2) Run this cell

import anndata as ad

adata = ad.read_h5ad(DATA_PATH, backed="r")  # lazy read; applying creates a copy by default
adata2 = viewer.apply_session_to_anndata(adata, inplace=False)
adata2